Praktikum D1

In [ ]:
import cv2, time

cap = cv2.VideoCapture(0)

frames, t0 = 0, time.time()
while True:
    ok, frame = cap.read()
    frames = frames + 1
    if time.time() - t0 >= 1.0 :
        cv2.setWindowTitle("Preview", f"Preview (FPS ~ {frames})")
        frames, t0 = 0, time.time()
    cv2.imshow("Webcam Preview", frame)
    
    cv2.waitKey(1)
    
    if cv2.waitKey(1) & 0xFF ==ord('q'):break
    
cap.release()
cv2.destroyAllWindows()

Praktikum D2

In [ ]:
import cv2 
from cvzone.PoseModule import PoseDetector

# Inisialisasi pengambilan video dari kamera (indeks 0)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    # Munculkan kesalahan jika kamera tidak dapat dibuka
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi PoseDetector
detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)

# Loop utama untuk memproses frame
while True:
    # Tangkap setiap frame dari webcam
    success, img = cap.read()
 
    # Periksa apakah frame berhasil ditangkap
    if not success:
        print("Gagal membaca frame dari kamera.")
        break

    # Temukan pose manusia dalam frame
    img = detector.findPose(img)

    # Temukan landmark (lmList) dan info bounding box (bboxInfo)
    lmList, bboxInfo = detector.findPosition(img, draw=True,
                                             bboxWithHands=False)

    # Periksa apakah ada landmark tubuh yang terdeteksi
    if lmList:
        # Dapatkan pusat bounding box
        center = bboxInfo["center"]

        # Gambar lingkaran di pusat bounding box
        cv2.circle(img, center, 5, (255, 0, 255), cv2.FILLED)

        # Hitung jarak antara landmark 11 (Bahu Kiri) dan 15 (Pergelangan Tangan Kiri)
        length, img, info = detector.findDistance(lmList[11][0:2],
                                                  lmList[15][0:2],
                                                  img=img,
                                                  color=(255, 0, 0),
                                                  scale=10)
                                                  
        print("Jarak: ", length)
        # Hitung sudut antara landmark 11 (Bahu Kiri), 13 (Siku Kiri), dan 15 (Pergelangan Tangan Kiri)
        angle, img = detector.findAngle(lmList[11][0:2],
                                        lmList[13][0:2],
                                        lmList[15][0:2],
                                        img=img,
                                        color=(0, 0, 255),
                                        scale=10)

        # Periksa apakah sudut mendekati 50 derajat dengan toleransi 10
        isCloseAngle50 = detector.angleCheck(myAngle=angle,
                                             targetAngle=50,
                                             offset=10)

        # Cetak hasil pemeriksaan sudut
        print(isCloseAngle50)

    # Tampilkan frame
    cv2.imshow("Pose + Angle ", img)

    # Keluar jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bebaskan sumber daya dan tutup jendela
cap.release()
cv2.destroyAllWindows()

Praktikum D3

In [ ]:
import cv2
import numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

# Indeks landmark mata kiri (berdasarkan MediaPipe Face Mesh)
# L_TOP (159): atas, L_BOTTOM (145): bawah
# L_LEFT (33): kiri, L_RIGHT (133): kanan
L_TOP, L_BOTTOM, L_LEFT, L_RIGHT = 159, 145, 33, 133

# Fungsi untuk menghitung jarak Euclidean antara dua titik (p1, p2)
def dist(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

# Inisialisasi pengambilan video dari kamera (indeks 0)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi objek FaceMeshDetector
detector = FaceMeshDetector(staticMode=False, maxFaces=2,
                            minDetectionCon=0.5, minTrackCon=0.5)

# Variabel untuk menghitung kedipan sederhana
blink_count = 0
closed_frames = 0
CLOSED_FRAMES_THRESHOLD = 3  # Jumlah frame berturut-turut untuk dianggap kedipan
EYE_AR_THRESHOLD = 0.20      # Ambang Eye Aspect Ratio (EAR) untuk menilai mata tertutup
is_closed = False            # Status apakah mata sudah dihitung sebagai 'tertutup'

# Loop utama untuk memproses frame
while True:
    ok, img = cap.read()
    if not ok: break

    # Temukan Face Mesh di dalam frame
    img, faces = detector.findFaceMesh(img, draw=True)

    if faces:
        # Ambil data mesh wajah pertama
        face = faces[0]  # list of 468 (x,y) landmarks

        # Hitung jarak vertikal (v) dan horizontal (h) mata kiri
        v = dist(face[L_TOP], face[L_BOTTOM])
        h = dist(face[L_LEFT], face[L_RIGHT])

        # Hitung Eye Aspect Ratio (EAR)
        # Menambahkan 1e-8 untuk menghindari pembagian dengan nol
        ear = v / (h + 1e-8)

        # Tampilkan nilai EAR pada frame
        cv2.putText(img, f"EAR(L): {ear:.3f}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        # Logika counter kedipan sederhana:
        if ear < EYE_AR_THRESHOLD:
            closed_frames += 1

            # Jika mata tertutup cukup lama dan belum dihitung sebagai kedipan
            if closed_frames >= CLOSED_FRAMES_THRESHOLD and not is_closed:
                blink_count += 1
                is_closed = True  # Tandai mata sudah dihitung sebagai tertutup/kedipan
        else:
            # Jika mata terbuka (EAR > EYE_AR_THRESHOLD)
            closed_frames = 0
            is_closed = False  # Setel ulang status

        # Tampilkan jumlah kedipan pada frame
        cv2.putText(img, f"Blink: {blink_count}", (20, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Tampilkan frame
    cv2.imshow("FaceMesh + EAR", img)

    # Keluar jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'): break

# Setelah loop berakhir, bebaskan sumber daya dan tutup jendela
cap.release()
cv2.destroyAllWindows()

Praktikum D4

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector

# Inisialisasi pengambilan video dari kamera (indeks 0)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi HandDetector
detector = HandDetector(staticMode=False, maxHands=1,
                        modelComplexity=1, detectionCon=0.5,
                        minTrackCon=0.5)

# Loop utama untuk memproses frame
while True:
    ok, img = cap.read()
    if not ok: break

    # Temukan tangan dalam frame
    hands, img = detector.findHands(img, draw=True, flipType=True) # flipType untuk mirror UI

    if hands:
        # Ambil data tangan pertama
        hand = hands[0] # dict berisi "lmList", "bbox", dll.

        # Tentukan jari mana yang terangkat
        fingers = detector.fingersUp(hand) # list panjang 5 berisi 0/1

        # Hitung total jari yang terangkat
        count = sum(fingers)

        # Tampilkan jumlah jari dan status fingersUp pada frame
        cv2.putText(img, f"Fingers: {count} {fingers}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Tampilkan frame
    cv2.imshow("Hands + Fingers", img)

    # Keluar jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'): break

# Setelah loop berakhir, bebaskan sumber daya dan tutup jendela
cap.release()
cv2.destroyAllWindows()

Praktikum D5

In [ ]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

# Fungsi untuk menghitung jarak Euclidean antara dua titik (a dan b)
def dist(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

# Fungsi utama untuk mengklasifikasikan gesture
def classify_gesture(hand):
    # hand["lmList"] berisi 21 landmark (x,y,z)
    lm = hand["lmList"]
    
    # Ambil koordinat (x,y) dari landmark kunci
    # Landmark: 0=Pergelangan Tangan, 4=Ujung Ibu Jari, 8=Ujung Jari Telunjuk, dst.
    wrist = np.array(lm[0][:2])
    thumb_tip = np.array(lm[4][:2])
    index_tip = np.array(lm[8][:2])
    middle_tip = np.array(lm[12][:2])
    ring_tip = np.array(lm[16][:2])
    pinky_tip = np.array(lm[20][:2])

    # Hitung rata-rata jarak relatif semua ujung jari ke pergelangan tangan (wrist)
    r_mean = np.mean([dist(index_tip, wrist), dist(middle_tip, wrist),
                      dist(ring_tip, wrist), dist(pinky_tip, wrist),
                      dist(thumb_tip, wrist)])

    # --- Aturan Klasifikasi ---

    # 1. Gesture "OK" (Ibu jari dan Telunjuk berdekatan)
    if dist(thumb_tip, index_tip) < 35: 
        return "OK"

    # 2. Gesture "THUMBS_UP"
    # Syarat: Ibu jari lebih tinggi (koordinat y lebih kecil) dari pergelangan tangan,
    # DAN jarak ibu jari ke pergelangan tangan relatif panjang.
    if (thumb_tip[1] < wrist[1] - 40) and \
       (dist(thumb_tip, wrist) > 0.8 * dist(index_tip, wrist)):
        return "THUMBS_UP"

    # 3. Gesture "ROCK" (Tangan mengepal)
    # Syarat: Rata-rata jarak ujung jari ke pergelangan tangan sangat kecil.
    if r_mean < 120: 
        return "ROCK"

    # 4. Gesture "PAPER" (Tangan terbuka)
    # Syarat: Rata-rata jarak ujung jari ke pergelangan tangan sangat besar.
    if r_mean > 200: 
        return "PAPER"

    # 5. Gesture "SCISSORS" (Jari Telunjuk dan Jari Tengah terangkat)
    # Syarat: Jarak Telunjuk dan Jari Tengah ke pergelangan tangan besar (terangkat),
    # DAN jarak Jari Manis dan Kelingking ke pergelangan tangan kecil (terlipat).
    if dist(index_tip, wrist) > 180 and dist(middle_tip, wrist) > 180 and \
       dist(ring_tip, wrist) < 160 and dist(pinky_tip, wrist) < 160:
        return "SCISSORS"

    # Jika tidak ada aturan di atas yang terpenuhi
    return "UNKNOWN"

# Inisialisasi pengambilan video dari kamera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi HandDetector
detector = HandDetector(staticMode=False, maxHands=1,
                        modelComplexity=1, detectionCon=0.5,
                        minTrackCon=0.5)

# Loop utama
while True:
    ok, img = cap.read()
    if not ok: break

    # Temukan tangan (flipType=True untuk tampilan yang nyaman)
    hands, img = detector.findHands(img, draw=True, flipType=True)

    if hands:
        # Klasifikasikan gesture tangan pertama
        label = classify_gesture(hands[0])
        
        # Tampilkan hasil klasifikasi pada frame
        cv2.putText(img, f"Gesture: {label}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    # Tampilkan frame
    cv2.imshow("Hand Gestures (cvzone)", img)

    # Keluar jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'): break

# Bebaskan sumber daya
cap.release()
cv2.destroyAllWindows()

Praktikum D6

In [ ]:
import cv2, numpy as np
from collections import deque
from cvzone.PoseModule import PoseDetector

MODE = "squat"   # tekan 'm' untuk toggle ke "pushup"
KNEE_DOWN, KNEE_UP = 80, 160     # ambang squat (deg)
DOWN_R, UP_R     = 0.85, 1.00    # ambang push-up (rasio)
SAMPLE_OK        = 4             # minimal frame konsisten sebelum ganti state

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)

count, state = 0, "up"
debounce = deque(maxlen=6)

def ratio_pushup(lm):
    # gunakan kiri: 11=shoulderL, 15=wristL, 23=hipL
    sh = np.array(lm[11][1:3])
    wr = np.array(lm[15][1:3])
    hp = np.array(lm[23][1:3])
    return np.linalg.norm(sh - wr) / (np.linalg.norm(sh - hp) + 1e-8)

while True:
    ok, img = cap.read()
    if not ok:
        break
    img = detector.findPose(img, draw=True)
    lmList, _ = detector.findPosition(img, draw=False)  # [(id,x,y,z,vis), ...]
    flag = None

    if lmList:
        if MODE == "squat":
            # rata-rata sudut lutut kiri & kanan
            # angL, _ = detector.findAngle(img, 23, 25, 27, draw=False)
            # angR, _ = detector.findAngle(img, 24, 26, 28, draw=False)

            angL, img = detector.findAngle(lmList[23][0:2],
                                           lmList[25][0:2],
                                           lmList[27][0:2],
                                           img=img,
                                           color=(0, 0, 255),
                                           scale=10)
            
            angR, img = detector.findAngle(lmList[24][0:2],
                                           lmList[26][0:2],
                                           lmList[28][0:2],
                                           img=img,
                                           color=(0, 255, 0),
                                           scale=10)

            ang = (angL + angR) / 2.0
            if ang < KNEE_DOWN:
                flag = "down"
            elif ang > KNEE_UP:
                flag = "up"
            cv2.putText(img, f"Knee: {ang:5.1f}", (20,70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        else:
            # push-up: rasio (shoulder–wrist)/(shoulder–hip)
            r = ratio_pushup(lmList)
            if r < DOWN_R:
                flag = "down"
            elif r > UP_R:
                flag = "up"
            cv2.putText(img, f"Ratio: {r:4.2f}", (20,70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

        debounce.append(flag)
        if debounce.count("down") >= SAMPLE_OK and state == "up":
            state = "down"
        if debounce.count("up") >= SAMPLE_OK and state == "down":
            state = "up"; count += 1

    cv2.putText(img, f"Mode: {MODE.upper()}  Count: {count}", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
    cv2.putText(img, f"State: {state}", (20,100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    cv2.imshow("Pose Counter", img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    if key == ord('m'):
        MODE = "pushup" if MODE == "squat" else "squat"

cap.release()
cv2.destroyAllWindows()
